In [2]:
import torch
from torch.nn import MSELoss
from torch.optim import Adam
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SchNet, GCNConv, global_mean_pool
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torch.utils.data import random_split
from torch_geometric.datasets import QM9

In [3]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler

## PREP

In [4]:
#root='data/QM9'
DATA_DIR = 'data/QM9'

In [5]:
class BasePreprocessor:
    """
    Base class for data preprocessing, handling common tasks like loading, 
    subsetting, splitting, and configuration.
    """
    def __init__(self, dataset_cls=QM9, root=DATA_DIR,  #data/QM9
                 transform=None, target=0, val_ratio=0.2,
                 seed=42, subset=None):

        self.dataset_cls = dataset_cls
        self.root = root
        self.transform = transform
        self.target = target
        self.val_ratio = val_ratio
        self.seed = seed
        self.subset = subset
        # Initialize internal storage for the dataset
        self._dataset = None

    # -------------------------
    # Loading dataset (Lazy-loaded)
    # -------------------------
    def _load_dataset(self):
        """Loads the dataset once and caches it."""
        if self._dataset is None:
            # Load the dataset (expensive I/O operation)
            dataset = self.dataset_cls(root=self.root, transform=self.transform)

            if self.subset:
                # Apply subset slicing
                dataset = dataset[:self.subset]
            
            self._dataset = dataset
            
        return self._dataset

    # -------------------------
    # Split train/val
    # -------------------------
    def split(self, processed):
        """Splits the processed dataset into train and validation sets."""
        n_val = int(len(processed) * self.val_ratio)
        # Ensure n_train is the remainder to cover the whole dataset
        n_train = len(processed) - n_val 

        gen = torch.Generator().manual_seed(self.seed)
        return random_split(processed, [n_train, n_val], generator=gen)

    # -------------------------
    # Abstract/Helper
    # -------------------------
    def _format_dataset(self, dataset, is_inference):
        """
        Abstract method for model-specific data formatting and target slicing.
        Must be implemented by subclasses.
          For MLP: extract z, and a column of y
          For GCN: extract z, pos, edge_index, y
          For SchNet: extract all fields
        """
        raise NotImplementedError

    # -------------------------
    # Concrete Workflow Methods (Handle Redundancy)
    # -------------------------
    
    def preprocess(self):
        """Workflow for train/validation: Load -> Format -> Split."""
        dataset = self._load_dataset()
        processed = self._format_dataset(dataset, is_inference=False)
        return self.split(processed)

    def preprocess_test(self):
        """Workflow for test: Load -> Format -> Return."""
        dataset = self._load_dataset()
        return self._format_dataset(dataset, is_inference=False)

    def preprocess_inference(self):
        """Workflow for inference: Load -> Format (no target) -> Return."""
        dataset = self._load_dataset()
        return self._format_dataset(dataset, is_inference=True)
    
    
    
class PreprocessorRegistry:
    _registry = {}

    @classmethod
    def register(cls, name):
        def decorator(prep_cls):
            cls._registry[name] = prep_cls
            return prep_cls
        return decorator

    @classmethod
    def create(cls, name, **kwargs):
        if name not in cls._registry:
            raise ValueError(f"Unknown preprocessor: {name}")
        return cls._registry[name](**kwargs)
    
    
@PreprocessorRegistry.register("mlp")
class MLPPreprocessor(BasePreprocessor):
    def __init__(self, **kwargs):
        # MLP doesn't need graph transforms
        super().__init__(transform=None, **kwargs)


    def _format_dataset(self, dataset, is_inference):
        """Extracts atomic numbers (z) and (optionally) the target (y)."""

        target_col = self.target

        out = []

        for d in dataset:
            d_new = d.clone()

            # remove unwanted fields
            for field in ["x", "edge_attr", "edge_index", "pos","name", "smiles", "idx"]:
                if hasattr(d_new, field):
                    delattr(d_new, field)

            # Explicitly set num_nodes
            d_new.num_nodes = d.z.size(0)

            # Handle inference/no-inference target
            if not is_inference:
                d_new.y = d.y[:, target_col].unsqueeze(1)
            else:
                if hasattr(d_new, "y"):
                    delattr(d_new, "y")

            out.append(d_new)

        return out
    
    
def loaders(batch_size, subset=10):

	model_type = 'mlp'


	prep = PreprocessorRegistry.create(
	    model_type,
	    target=0,
	    root=DATA_DIR,
	    subset=subset,
	)

	train_ds, val_ds = prep.preprocess()
	train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
	val_loader = DataLoader(val_ds, batch_size=batch_size)
	return train_loader, val_loader

## first model

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, num_atom_types=100, hidden=64):
        super().__init__()
        self.emb = nn.Embedding(num_atom_types, hidden)
        self.fc = nn.Sequential(
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )
    def forward(self, batch):
        x = self.emb(batch.z)       # [num_nodes, hidden]
        x = global_mean_pool(x, batch.batch)  # [num_graphs, hidden]
        out = self.fc(x)            # [num_graphs, 1]
        return out



def run_epoch(loader, model, criterion, optimizer=None):
    model.train() if optimizer else model.eval()
    total_loss = 0

    for batch in loader:
        batch = batch.to(device)
        out = model(batch)
        pred = out.view(-1)
        target = batch.y.view(-1)
        loss = criterion(pred, target)

        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * batch.num_graphs

    return total_loss / len(loader.dataset)



model_mlp = SimpleMLP().to(device)
optimizer = Adam(model_mlp.parameters(), lr=1e-3)
criterion = MSELoss()

# loaders
train_loader, val_loader = loaders(32)


# Train for a few epochs
for epoch in range(3):
    train_loss = run_epoch(train_loader, model_mlp, criterion, optimizer)
    val_loss = run_epoch(val_loader, model_mlp, criterion)
    print(f"[MLP] Epoch {epoch} | Train {train_loss:.4f} | Val {val_loss:.4f}")

## <font color='purple'> Tuning </font>

In [ ]:


def train_ray(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # small subset for speed
    subset = config["subset"]

    train_loader, val_loader = loaders(
        batch_size=config["batch_size"],
        subset=subset
    )

    model = SimpleMLP(
        num_atom_types=100,
        hidden=config["hidden"]
    ).to(device)

    optimizer = Adam(model.parameters(), lr=config["lr"])
    criterion = MSELoss()

    for epoch in range(config["epochs"]):
        train_loss = run_epoch(train_loader, model, criterion, optimizer)
        val_loss = run_epoch(val_loader, model, criterion)

        # report to Ray Tune
        tune.report({"val_loss":val_loss})


        
        
search_space = {
    "hidden": tune.choice([32, 64, 128]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "batch_size": tune.choice([16, 32]),
    "epochs": 3,
    "subset": 10,        # only 500 samples → very fast tuning
}


scheduler = ASHAScheduler(
    max_t=5,
    grace_period=1,
    reduction_factor=2,
)

tuner = tune.Tuner(
    train_ray,
    tune_config=tune.TuneConfig(
        metric="val_loss",
        mode="min",
        scheduler=scheduler,
        num_samples=2,   # small number of trials
    ),
    param_space=search_space,
)

results = tuner.fit()
best = results.get_best_result("val_loss", "min")
print("Best config:", best.config)


In [ ]:
results

## GCN

In [15]:
from torch_geometric.transforms import RadiusGraph

In [20]:
def run_epoch(loader, model, criterion, optimizer=None):
    model.train() if optimizer else model.eval()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        out = model(batch)
        pred = out.squeeze(-1)
        target = batch.y.squeeze(-1)
        loss = criterion(pred, target)
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    return total_loss / len(loader.dataset)

In [21]:
@PreprocessorRegistry.register("gcn")
class GCNPreprocessor(BasePreprocessor):
    def __init__(self, **kwargs):
        # Get user-supplied transform or use the default RadiusGraph
        radius = kwargs.pop("radius", 1.5)   # extracted only for GCN
        transform = RadiusGraph(r=radius)
        super().__init__(transform=transform, **kwargs)



    def _format_dataset(self, dataset, is_inference):
        """Extracts atomic numbers (z) and (optionally) the target (y)."""

        target_col = self.target

        out = []

        for d in dataset:
            d_new = d.clone()

            # remove unwanted fields
            for field in ["x", "edge_attr", "name", "smiles", "idx"]:
                if hasattr(d_new, field):
                    delattr(d_new, field)

            # Handle inference/no-inference target
            if not is_inference:
                d_new.y = d.y[:, target_col].unsqueeze(1)
            else:
                if hasattr(d_new, "y"):
                    delattr(d_new, "y")

            out.append(d_new)

        return out


In [22]:
class SimpleGCN(nn.Module):
    def __init__(self, hidden=64, num_atom_types=100):
        super().__init__()
        self.emb = nn.Embedding(num_atom_types, hidden)
        self.conv1 = GCNConv(hidden, hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, batch):
        x = self.emb(batch.z)           # [num_nodes, hidden]
        x = self.conv1(x, batch.edge_index) # using edge_index here
        x = torch.relu(x)
        x = self.conv2(x, batch.edge_index)
        x = torch.relu(x)
        x = global_mean_pool(x, batch.batch)  # [num_graphs, hidden]
        out = self.fc(x)                        # [num_graphs, 1]
        return out


In [23]:
def loadersGCN(batch_size, subset=10):

	model_type = 'gcn'


	prep = PreprocessorRegistry.create(
	    model_type,
	    target=0,
	    root=DATA_DIR,
	    subset=subset,
	)
    

	train_ds, val_ds = prep.preprocess() # -> this result goers to ray.put
    
    
	train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
	val_loader = DataLoader(val_ds, batch_size=batch_size)
	return train_loader, val_loader

In [24]:


def train_ray(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # small subset for speed
    subset = config["subset"]

    train_loader, val_loader = loadersGCN(
        batch_size=config["batch_size"],
        subset=subset
    )

    model = SimpleGCN(
        num_atom_types=100,
        hidden=config["hidden"]
    ).to(device)

    optimizer = Adam(model.parameters(), lr=config["lr"])
    criterion = MSELoss()

    for epoch in range(config["epochs"]):
        train_loss = run_epoch(train_loader, model, criterion, optimizer)
        val_loss = run_epoch(val_loader, model, criterion)

        # report to Ray Tune
        tune.report({"val_loss":val_loss})


        
        
search_space = {
    "hidden": tune.choice([32, 64, 128]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "batch_size": tune.choice([16, 32]),
    "epochs": 3,
    "subset": 10,        # only 500 samples → very fast tuning
}


scheduler = ASHAScheduler(
    max_t=5,
    grace_period=1,
    reduction_factor=2,
)

tuner = tune.Tuner(
    train_ray,
    tune_config=tune.TuneConfig(
        metric="val_loss",
        mode="min",
        scheduler=scheduler,
        num_samples=2,   # small number of trials
    ),
    param_space=search_space,
)

results = tuner.fit()
best = results.get_best_result("val_loss", "min")
print("Best config:", best.config)


(train_ray pid=10415) Downloading https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/molnet_publish/qm9.zip
(train_ray pid=10414) Extracting data/QM9/raw/qm9.zip
(train_ray pid=10414) Downloading https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/molnet_publish/qm9.zip
(train_ray pid=10414) Downloading https://ndownloader.figshare.com/files/3195404
(train_ray pid=10414) Processing...
  0%|          | 568/133885 [00:00<01:38, 1349.77it/s]
(train_ray pid=10415) Extracting data/QM9/raw/qm9.zip
  1%|▏         | 1802/133885 [00:01<01:41, 1296.28it/s]
(train_ray pid=10415) Downloading https://ndownloader.figshare.com/files/3195404
  3%|▎         | 4607/133885 [00:03<01:42, 1266.32it/s]
(train_ray pid=10415) Processing...
 94%|█████████▍| 126295/133885 [02:02<00:06, 1145.73it/s]


 98%|█████████▊| 131189/133885 [02:08<00:02, 1122.52it/s]
(train_ray pid=10414) Done!
 99%|█████████▊| 132189/133885 [02:09<00:01, 1312.57it/s]


100%|██████████| 133885/133885 [02:10<00:00, 1024.00it/s]
(train_ray pid=10415) Done!
2025-11-21 16:38:22,289	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/marcos/ray_results/train_ray_2025-11-21_16-35-39' in 0.0040s.
2025-11-21 16:38:22,295	INFO tune.py:1041 -- Total run time: 163.03 seconds (162.99 seconds for the tuning loop).


Best config: {'hidden': 64, 'lr': 0.001425497839559118, 'batch_size': 32, 'epochs': 3, 'subset': 10}


In [25]:
results

ResultGrid<[
  Result(
    metrics={'val_loss': 0.5780990719795227},
    path='/home/marcos/ray_results/train_ray_2025-11-21_16-35-39/train_ray_42844_00000_0_batch_size=32,hidden=64,lr=0.0014_2025-11-21_16-35-39',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'val_loss': 0.7820579409599304},
    path='/home/marcos/ray_results/train_ray_2025-11-21_16-35-39/train_ray_42844_00001_1_batch_size=32,hidden=64,lr=0.0025_2025-11-21_16-35-39',
    filesystem='local',
    checkpoint=None
  )
]>

### Schnet

In [26]:
@PreprocessorRegistry.register("schnet")
class SchNetPreprocessor(BasePreprocessor):
    def __init__(self, **kwargs):
        # SchNet often handles connectivity internally (e.g., using radius graph
        # and pos), so we might not need an explicit transform here, depending on 
        # how the model is implemented. Sticking with None for consistency.
        cutoff = kwargs.pop("cutoff", 10.0)
        super().__init__(transform=None, **kwargs)
        
    
    def _format_dataset(self, dataset, is_inference):
        target_col = self.target

        processed = []
        for d in dataset:
            d_new = d.clone()  # <-- keeps all fields: pos, z, edge_index, etc

            if is_inference:
                # remove y safely if exists
                if hasattr(d_new, "y"):
                    del d_new.y
            else:
                d_new.y = d.y[:, target_col].unsqueeze(1)

            processed.append(d_new)

        return processed

In [38]:
def loaders_schnet(batch_size, subset=10):

	model_type = 'schnet'


	prep = PreprocessorRegistry.create(
	    model_type,
	    target=0,
	    root=DATA_DIR,
	    subset=subset,
	)
    

	train_ds, val_ds = prep.preprocess() # -> this result goers to ray.put
    
    
	train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
	val_loader = DataLoader(val_ds, batch_size=batch_size)
	return train_loader, val_loader

In [34]:
import torch
import torch.nn as nn
from torch_geometric.nn import SchNet

class TunableSchNet(nn.Module):
    def __init__(self, hidden_channels, num_filters, num_interactions,
                 num_gaussians, cutoff):
        super().__init__()

        self.model = SchNet(
            hidden_channels=hidden_channels,
            num_filters=num_filters,
            num_interactions=num_interactions,
            num_gaussians=num_gaussians,
            cutoff=cutoff,
            readout='add'
        )

        # IMPORTANT: output dim = hidden_channels
        self.regressor = nn.Linear(hidden_channels, 1)

    def forward(self, z, pos, batch):
        x = self.model(z, pos, batch)   # graph embeddings
        return self.regressor(x).squeeze(-1)

    
    
def run_epoch(loader, model, criterion, optimizer=None, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total = 0
    for batch in loader:
        batch = batch.to(model.regressor.weight.device)

        pred = model(batch.z, batch.pos, batch.batch)
        loss = criterion(pred, batch.y)

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total += float(loss)

    return total / len(loader)



def train_ray(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # small subset for speed
    subset = config["subset"]

    train_loader, val_loader = loadersSchNet(
        batch_size=config["batch_size"],
        subset=subset
    )

    model = TunableSchNet(
        hidden_channels=config["hidden_channels"],
        num_filters=config["num_filters"],
        num_interactions=config["num_interactions"],
        num_gaussians=config["num_gaussians"],
        cutoff=config["cutoff"]
    ).to(device)
    
    

    
    optimizer = Adam(list(model.parameters()), lr=config["lr"])
    criterion = MSELoss()



    for epoch in range(1, 11):
        train_loss = run_epoch(train_loader, model, criterion, optimizer, train=True)
        val_loss = run_epoch(val_loader, model, criterion, optimizer, train=False)
        print(f"Epoch {epoch:02d} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

        # report to Ray Tune
        tune.report({"val_loss":val_loss})


        
        
search_space = {
    "hidden_channels": tune.choice([64, 128, 256]),
    "num_filters": tune.choice([64, 128, 256]),
    "num_interactions": tune.choice([3, 4, 6]),
    "num_gaussians": tune.choice([25, 50, 75]),
    "cutoff": tune.choice([5.0, 6.0, 8.0, 10.0]),

    # training hyperparams
    "lr": tune.loguniform(1e-4, 3e-3),
    "batch_size": tune.choice([16, 32]),
    "epochs": 3,
    "subset": 10,
}


scheduler = ASHAScheduler(
    max_t=5,
    grace_period=1,
    reduction_factor=2,
)

tuner = tune.Tuner(
    train_ray,
    tune_config=tune.TuneConfig(
        metric="val_loss",
        mode="min",
        scheduler=scheduler,
        num_samples=2,   # small number of trials
    ),
    param_space=search_space,
)

results = tuner.fit()
best = results.get_best_result("val_loss", "min")
print("Best config:", best.config)



(train_ray pid=12307) Downloading https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/molnet_publish/qm9.zip
(train_ray pid=12307) Extracting data/QM9/raw/qm9.zip
(train_ray pid=12306) Downloading https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/molnet_publish/qm9.zip
(train_ray pid=12307) Downloading https://ndownloader.figshare.com/files/3195404
(train_ray pid=12307) Processing...
  0%|          | 437/133885 [00:00<01:35, 1395.11it/s]
(train_ray pid=12306) Extracting data/QM9/raw/qm9.zip
  1%|▏         | 1769/133885 [00:01<01:45, 1256.41it/s]
(train_ray pid=12306) Downloading https://ndownloader.figshare.com/files/3195404
  2%|▏         | 2501/133885 [00:02<01:57, 1114.18it/s]
(train_ray pid=12306) Processing...
 91%|█████████▏| 122496/133885 [02:07<00:12, 920.33it/s]


 99%|█████████▊| 132096/133885 [02:16<00:01, 1069.69it/s]
(train_ray pid=12307) Done!
 99%|█████████▉| 132573/133885 [02:17<00:01, 1140.98it/s]
2025-11-21 17:05:47,074	ERROR tune_controller.py:1331 -- Trial task failed for trial train_ray_118ca_00001
Traceback (most recent call last):
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2858, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/wor

100%|██████████| 133885/133885 [02:18<00:00, 969.57it/s] 
2025-11-21 17:05:51,891	ERROR tune_controller.py:1331 -- Trial task failed for trial train_ray_118ca_00000
Traceback (most recent call last):
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2858, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "/home/marcos/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 958, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions

RuntimeError: No best trial found for the given metric: val_loss. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [45]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import MSELoss
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SchNet

from ray import tune
from ray.tune.schedulers import ASHAScheduler

# ============================================================
#                   Tunable SchNet Model
# ============================================================

class TunableSchNet(nn.Module):
    def __init__(self, hidden_channels, num_filters,
                 num_interactions, num_gaussians, cutoff):
        super().__init__()

        self.model = SchNet(
            hidden_channels=hidden_channels,
            num_filters=num_filters,
            num_interactions=num_interactions,
            num_gaussians=num_gaussians,
            cutoff=cutoff,
            readout='add'
        )

        # keep regressor but use conditionally
        self.regressor = nn.Linear(hidden_channels, 1)

    def forward(self, batch):
        # call SchNet with explicit z, pos, batch
        x = self.model(batch.z, batch.pos, batch.batch)

        # possible outputs:
        # - shape [B, hidden_channels]  -> apply regressor
        # - shape [B, 1] or [B]         -> already scalar per graph
        if x.dim() == 1:                # [B]
            return x
        if x.dim() == 2 and x.size(1) == 1:  # [B,1]
            return x.view(-1)
        # otherwise assume [B, hidden_channels]
        return self.regressor(x).squeeze(-1)

        

# ============================================================
#                   Dataset Loader
# ============================================================



# ============================================================
#                   Training Helpers
# ============================================================

def run_epoch(loader, model, criterion, optimizer=None, train=True):
    model.train() if train else model.eval()

    total = 0
    for batch in loader:
        batch = batch.to(next(model.parameters()).device)

        pred = model(batch)             
        #loss = criterion(pred, batch.y)
        target = batch.y.view(-1)   # [B]
        loss = criterion(pred, target)


        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total += float(loss)

    return total / len(loader)


# ============================================================
#                   Ray Tune Trainable
# ============================================================

def train_ray(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # loader size is part of config
    train_loader, val_loader = loaders_schnet(
        batch_size=config["batch_size"],
        subset=config["subset"]
    )

    model = TunableSchNet(
        hidden_channels=config["hidden_channels"],
        num_filters=config["num_filters"],
        num_interactions=config["num_interactions"],
        num_gaussians=config["num_gaussians"],
        cutoff=config["cutoff"]
    ).to(device)

    optimizer = Adam(model.parameters(), lr=config["lr"])
    criterion = MSELoss()

    # train loop
    for epoch in range(config["epochs"]):
        train_loss = run_epoch(train_loader, model, criterion, optimizer, train=True)
        val_loss = run_epoch(val_loader, model, criterion, train=False)

        # must report for Ray
        tune.report({"val_loss":val_loss})


# ============================================================
#                   Search Space + Tuner
# ============================================================

search_space = {
    # SchNet architecture
    "hidden_channels": tune.choice([64, 128, 256]),
    "num_filters": tune.choice([64, 128, 256]),
    "num_interactions": tune.choice([3, 4, 6]),
    "num_gaussians": tune.choice([25, 50, 75]),
    "cutoff": tune.choice([5.0, 6.0, 8.0, 10.0]),

    # Training hyperparams
    "lr": tune.loguniform(1e-4, 3e-3),
    "batch_size": tune.choice([16, 32]),
    "epochs": 2,
    "subset": 10,
}

scheduler = ASHAScheduler(
    max_t=5,
    grace_period=1,
    reduction_factor=2,
)

tuner = tune.Tuner(
    train_ray,
    tune_config=tune.TuneConfig(
        metric="val_loss",
        mode="min",
        scheduler=scheduler,
        num_samples=1,
    ),
    param_space=search_space,
)

results = tuner.fit()

best = results.get_best_result(metric="val_loss", mode="min")
print("\nBest config:")
print(best.config)


(train_ray pid=15421) Downloading https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/molnet_publish/qm9.zip
(train_ray pid=15421) Extracting data/QM9/raw/qm9.zip
(train_ray pid=15421) Downloading https://ndownloader.figshare.com/files/3195404
(train_ray pid=15421) Processing...
 13%|█▎        | 16811/133885 [00:15<01:49, 1069.78it/s]


 25%|██▌       | 34110/133885 [00:30<01:36, 1034.57it/s]


 37%|███▋      | 49267/133885 [00:45<01:20, 1045.49it/s]


 48%|████▊     | 64849/133885 [01:00<01:07, 1023.05it/s]


 60%|█████▉    | 79980/133885 [01:16<00:57, 939.56it/s]


 71%|███████   | 94708/133885 [01:31<00:42, 919.66it/s]


 82%|████████▏ | 109468/133885 [01:47<00:25, 953.40it/s]


 93%|█████████▎| 124008/133885 [02:02<00:08, 1125.69it/s]


100%|██████████| 133885/133885 [02:10<00:00, 1024.50it/s]
(train_ray pid=15421) Done!
2025-11-21 17:37:44,917	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/marcos/ray_results/train_ray_2025-11-21_17-35-02' in 0.0035s.
2025-11-21 17:37:44,924	INFO tune.py:1041 -- Total run time: 162.39 seconds (162.36 seconds for the tuning loop).



Best config:
{'hidden_channels': 256, 'num_filters': 64, 'num_interactions': 4, 'num_gaussians': 50, 'cutoff': 5.0, 'lr': 0.0002842349613009927, 'batch_size': 16, 'epochs': 2, 'subset': 10}


In [46]:
results

ResultGrid<[
  Result(
    metrics={'val_loss': 1.7692179679870605},
    path='/home/marcos/ray_results/train_ray_2025-11-21_17-35-02/train_ray_8e635_00000_0_batch_size=16,cutoff=5.0000,hidden_channels=256,lr=0.0003,num_filters=64,num_gaussians=50,num_interactions_2025-11-21_17-35-02',
    filesystem='local',
    checkpoint=None
  )
]>

In [ ]:
"""
from torch import nn
from torch_geometric.nn.models.schnet import SchNet

class TunableSchNet(nn.Module):
    def __init__(
        self,
        hidden_channels=64,
        num_filters=64,
        num_interactions=3,
        num_gaussians=50,
        cutoff=10.0,
        readout="add"
    ):
        super().__init__()

        self.model = SchNet(
            hidden_channels=hidden_channels,
            num_filters=num_filters,
            num_interactions=num_interactions,
            num_gaussians=num_gaussians,
            cutoff=cutoff,
            readout=readout
        )

        # SchNet outputs graph embeddings of size hidden_channels
        self.regressor = nn.Linear(hidden_channels, 1)

    def forward(self, batch):
        x = self.model(batch)          # [num_graphs, hidden]
        out = self.regressor(x)        # [num_graphs, 1]
        return out




search_space = {
    "hidden_channels": tune.choice([64, 128, 256]),
    "num_filters": tune.choice([64, 128, 256]),
    "num_interactions": tune.choice([3, 4, 6]),
    "num_gaussians": tune.choice([25, 50, 75]),
    "cutoff": tune.choice([5.0, 6.0, 8.0, 10.0]),

    # training hyperparams
    "lr": tune.loguniform(1e-4, 3e-3),
    "batch_size": tune.choice([16, 32]),
    "epochs": 3,
    "subset": 10,
}


def train_ray(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    train_loader, val_loader = loadersGCN(
        batch_size=config["batch_size"],
        subset=config["subset"]
    )

    model = TunableSchNet(
        hidden_channels=config["hidden_channels"],
        num_filters=config["num_filters"],
        num_interactions=config["num_interactions"],
        num_gaussians=config["num_gaussians"],
        cutoff=config["cutoff"]
    ).to(device)

    optimizer = Adam(model.parameters(), lr=config["lr"])
    criterion = MSELoss()

    for epoch in range(config["epochs"]):
        train_loss = run_epoch(train_loader, model, criterion, optimizer)
        val_loss = run_epoch(val_loader, model, criterion)

        tune.report({"val_loss": val_loss})


"""

now, withgout downlading data everytime

In [ ]:

# ---- GLOBAL PRELOAD (runs once) ----
"""
prep = PreprocessorRegistry.create("mlp", root="data/QM9", subset=50)
dataset = prep.preprocess()          # cached on disk after first run

def loadersv2(batch_size=32, subset=None):
    global dataset                 # ← reuse the preloaded dataset

    ds = dataset
    if subset is not None:
        ds = ds[:subset]

    train_ds, val_ds = split(ds)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)
    return train_loader, val_loader

"""
